# Mango Classification exploration

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from  fastai.importsfastai.  import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [ ]:
torchtorch..cudacuda..set_deviceset_dev (0)

In [ ]:
PATH = "data/mango/"
sz = 224
arch = resnext101_64
bs = 58

In [ ]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1 # header is not counted (-1)
val_idxs = get_cv_idxs(n) # random 20% data for validation set

In [ ]:
n

In [2]:
len(val_idxs)

NameError: name 'val_idxs' is not defined

In [ ]:
# If you haven't downloaded weights.tgz yet, download the file.
#     http://forums.fast.ai/t/error-when-trying-to-use-resnext50/7555
#     http://forums.fast.ai/t/lesson-2-in-class-discussion/7452/222
#!wget -O fastai/weights.tgz http://files.fast.ai/models/weights.tgz

#!tar xvfz fastai/weights.tgz -C fastai

# Initial Exploration

In [ ]:
!ls {PATH}

In [ ]:
label_df = pd.read_csv(label_csv)

In [ ]:
label_df.head()

In [ ]:
# test from gpu - did this push